Hi there,

I here have tried to do some exploratory data analysis and create visualizations for the same.
Hope they help you as well in trying to understand the dataset we have for this competition.


**Hope you gain some insights from it and find it useful! Do upvote and share it if you like it! :)**


Some of the basics first.

**Competition Name**        : Tabular Playground Series - May 2021
**Competition Description** : The dataset is used for this competition is synthetic, but based on a real dataset and generated using a CTGAN. The original dataset deals with predicting the category on an eCommerce product given various attributes about the listing. Although the features are anonymized, they have properties relating to real-world features.
**Competition evaluation Metric** : Submissions are evaluated using multi-class logarithmic loss. Each row in the dataset has been labeled with one true Class. For each row, you must submit the predicted probabilities that the product belongs to each class label. The formula is:

**log loss = −1/N ∑∑ yijlog(pij)**


where N is the number of rows in the test set, M is the number of class labels, log is the natural logarithm, yij is 1 if observation i is in class j and 0 otherwise, and pij is the predicted probability that observation i belongs to class j.

The submitted probabilities for a given product are not required to sum to one; they are rescaled prior to being scored, each row is divided by the row sum. In order to avoid the extremes of the log function, predicted probabilities are replaced with max(min(p,1−10−15),10−15).

**Competition Timeline** : 
- Start Date - May 1, 2021
- Entry deadline - Same as the Final Submission Deadline
- Team Merger deadline - Same as the Final Submission Deadline
- Final submission deadline - May 31, 2021

Points to note:
- This competition does not award ranking points
- This competition does not count towards tiers

**The kernel will have 5 parts:**
- Basic Introduction
- EDA
- Plots
- Model
- SHAP

# Basic Introduction :

Categorical data is a collection of information that is divided into groups. I.e, if an organisation or agency is trying to get a biodata of its employees, the resulting data is referred to as categorical

**In statistics, a categorical variable is a variable that can take on one of a limited, and usually fixed, number of possible values, assigning each individual or other unit of observation to a particular group or nominal category on the basis of some qualitative property**

Machine learning and deep learning models, like those in Keras, require all input and output variables to be numeric.

This means that if your data contains categorical data, you must encode it to numbers before you can fit and evaluate a model.

The two most popular techniques are an integer encoding and a one hot encoding, although a newer technique called learned embedding may provide a useful middle ground between these two methods.



**The Challenge With Categorical Data**
A categorical variable is a variable whose values take on the value of labels.

For example, the variable may be “color” and may take on the values “red,” “green,” and “blue.”

Sometimes, the categorical data may have an ordered relationship between the categories, such as “first,” “second,” and “third.” This type of categorical data is referred to as ordinal and the additional ordering information can be useful.

Machine learning algorithms and deep learning neural networks require that input and output variables are numbers.

This means that categorical data must be encoded to numbers before we can use it to fit and evaluate a model.

There are many ways to encode categorical variables for modeling, although the three most common are as follows:

- **Integer Encoding**: Where each unique label is mapped to an integer.
- **One Hot Encoding**: Where each label is mapped to a binary vector.
- **Learned Embedding**: Where a distributed representation of the categories is learned.

# Importing the data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# !pip install pycomp
from pandas_datareader import data

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print('Rows and Columns in train dataset:', train_df.shape)
print('Rows and Columns in test dataset:', test_df.shape)
print()
print('Missing values in train dataset:', sum(train_df.isnull().sum()))
print('Missing values in test dataset:', sum(test_df.isnull().sum()))
print()
print('Missing values per columns in train dataset')
for col in train_df.columns:
    temp_col = train_df[col].isnull().sum()
    print(f'{col}: {temp_col}')
print()
print('Missing values per columns in test dataset')
for col in test_df.columns:
    temp_col = test_df[col].isnull().sum()
    print(f'{col}: {temp_col}')
    

In [ ]:
train_df.head(1)

# Data Overview
Oberservations after having a quick look at the data:

- 100,000 data points of type integer
- 50 features (feature_0 through feature_49)
- 1 target
- 0 missing values


Categorical Features:

A categorical variable is one that has two or more categories and each value in that feature can be categorised by them.For example, gender is a categorical variable having two categories (male and female). Now we cannot sort or give any ordering to such variables. They are also known as Nominal Variables.

Ordinal Features:

An ordinal variable is similar to categorical values, but the difference between them is that we can have relative ordering or sorting between the values. For eg: If we have a feature like Height with values Tall, Medium, Short, then Height is a ordinal variable. Here we can have a relative sort in the variable

# Analysis
The first thing we are going to check is the distribution of the target feature. It's important to know if the class is balanced or not. If so, we would probably have to handle it.

In [ ]:
plt.rcParams['figure.dpi'] = 300
fig = plt.figure(figsize=(5, 5), facecolor='#f6f5f5')
gs = fig.add_gridspec(3, 2)
gs.update(wspace=0.4, hspace=0.8)

background_color = "#f6f5f5"

# background_color = "#f6f5f5"
column = 'target'
color_map = ['#eeb977', 'lightgray','plum','paleturquoise']
sns.set_palette(sns.color_palette(color_map))
temp_train = pd.DataFrame(train_df[column].value_counts()).reset_index(drop=False)
ax0 = fig.add_subplot(gs[0, 0])
for s in ["right", "top"]:
    ax0.spines[s].set_visible(False)
ax0.set_facecolor(background_color)
ax0.tick_params(axis = "y", which = "both", left = False)
ax0.text(-1, 83, 'Target distribution on the training data', color='black', fontsize=7, ha='left', va='bottom', weight='bold')
# ax0.text(-1, 82, 'Survival Rate ', color='#292929', fontsize=5, ha='left', va='top')
# ax0.text(1.18, 73.3, 'for age and fare', color='#292929', fontsize=4, ha='left', va='bottom')
ax0_sns = sns.barplot(ax=ax0, x=temp_train['index'], y=temp_train[column]/1000, zorder=2)
ax0_sns.set_xlabel("Target",fontsize=5, weight='bold')
ax0_sns.set_ylabel('')
ax0.yaxis.set_major_formatter(ticker.PercentFormatter())
ax0_sns.grid(which='major', axis='x', zorder=0, color='#EEEEEE')
ax0_sns.grid(which='major', axis='y', zorder=0, color='#EEEEEE')
ax0_sns.tick_params(labelsize=5)
ax0_sns.legend(['Class 1','Class 2','Class 3','Class 4'], ncol=2, facecolor=background_color, edgecolor=background_color, fontsize=4, bbox_to_anchor=(-0.26, 1.3), loc='upper left')
leg = ax0_sns.get_legend()
leg.legendHandles[0].set_color('#eeb977')
leg.legendHandles[1].set_color('lightgray')
leg.legendHandles[2].set_color('plum')
leg.legendHandles[3].set_color('paleturquoise')

**Observations :**
In the target variable, class2 has more data points compared to the remaining labels.so, we probably have to address class imbalance problem.

**Data Distibution**
Now lets look at the statistical distribution for the Training and the Test data:


In [ ]:
print("Training Data Statistical Distributions")

train_df.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

In [ ]:
print("Test Data Statistical Distributions")

test_df.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

**Observations:**
- No missing data
- The scale of the mean and deviation varies.
- The median is mostly 0, and there are 2 columns with median 1.
- The representative statistics of train and test are almost similar.
- Features are highly skewed.Applying feature transformations can help us build a better model.

In [ ]:
# feature_13
# feature_36
# feature_22

print("Features with Categories less than 10")
print()

color_map = ['#eeb977', 'lightgray','plum','paleturquoise']
plt.rcParams['figure.dpi'] = 300
fig = plt.figure(figsize=(5, 5), facecolor='#f6f5f5')
gs = fig.add_gridspec(4, 3)
gs.update(wspace=0.4, hspace=0.8)

background_color = "#f6f5f5"

column = 'feature_2'
# color_map = ['#eeb977', 'lightgray']
sns.set_palette(sns.color_palette(color_map))
temp_train = pd.DataFrame(train_df[column].value_counts()).reset_index(drop=False)
ax0 = fig.add_subplot(gs[0, 0])
for s in ["right", "top"]:
    ax0.spines[s].set_visible(False)
ax0.set_facecolor(background_color)
ax0.tick_params(axis = "y", which = "both", left = False)
# ax0.text(-1, 83, 'Survival Rate', color='black', fontsize=7, ha='left', va='bottom', weight='bold')
# ax0.text(-1, 82, 'feature_2 ', color='#292929', fontsize=5, ha='left', va='top')
# ax0.text(1.18, 73.3, 'for age and fare', color='#292929', fontsize=4, ha='left', va='top')
ax0_sns = sns.barplot(ax=ax0, x=temp_train['index'], y=temp_train[column]/1000, zorder=2)
ax0_sns.set_xlabel("feature_2",fontsize=5, weight='bold')
ax0_sns.set_ylabel('')
ax0.yaxis.set_major_formatter(ticker.PercentFormatter())
ax0_sns.grid(which='major', axis='x', zorder=0, color='#EEEEEE')
ax0_sns.grid(which='major', axis='y', zorder=0, color='#EEEEEE')
ax0_sns.tick_params(labelsize=5)
# ax0_sns.legend(['Survived', 'Not Survived'], ncol=2, facecolor=background_color, edgecolor=background_color, fontsize=4, bbox_to_anchor=(-0.26, 1.3), loc='upper left')
# leg = ax0_sns.get_legend()
# leg.legendHandles[0].set_color('#eeb977')
# leg.legendHandles[1].set_color('lightgray')

column = 'feature_13'
# color_map = ['#eeb977', 'lightgray', 'lightgray']
sns.set_palette(sns.color_palette(color_map))
temp_train = train_df.groupby(column)['feature_13'].sum()
ax1 = fig.add_subplot(gs[0, 1])
for s in ["right", "top"]:
    ax1.spines[s].set_visible(False)
ax1.set_facecolor(background_color)
ax1.tick_params(axis = "y", which = "both", left = False)
ax1_sns = sns.barplot(ax=ax1, x=temp_train.index, y=temp_train/1000, zorder=2)
ax1_sns.set_xlabel("feature_13",fontsize=5, weight='bold')
ax1_sns.set_ylabel('')
ax1.yaxis.set_major_formatter(ticker.PercentFormatter())
ax1_sns.grid(which='major', axis='x', zorder=0, color='#EEEEEE')
ax1_sns.grid(which='major', axis='y', zorder=0, color='#EEEEEE')
ax1_sns.tick_params(labelsize=5)

column = 'feature_36'
# color_map = ['#eeb977', 'lightgray']
sns.set_palette(sns.color_palette(color_map))
temp_train = train_df.groupby(column)['feature_36'].sum()
ax2 = fig.add_subplot(gs[0, 2])
for s in ["right", "top"]:
    ax2.spines[s].set_visible(False)
ax2.set_facecolor(background_color)
ax2.tick_params(axis = "y", which = "both", left = False)
ax2_sns = sns.barplot(ax=ax2, x=temp_train.index, y=temp_train/1000, zorder=2)
ax2_sns.set_xlabel("feature_36",fontsize=5, weight='bold')
ax2_sns.set_ylabel('')
ax2.yaxis.set_major_formatter(ticker.PercentFormatter())
ax2_sns.grid(which='major', axis='x', zorder=0, color='#EEEEEE')
ax2_sns.grid(which='major', axis='y', zorder=0, color='#EEEEEE')
ax2_sns.tick_params(labelsize=5)


column = 'feature_22'
# color_map = ['lightgray' for _ in range(7)]
color_map[0] = '#eeb977'
sns.set_palette(sns.color_palette(color_map))
temp_train = train_df.groupby(column)['feature_22'].sum()
ax4 = fig.add_subplot(gs[1, 1])
for s in ["right", "top"]:
    ax4.spines[s].set_visible(False)
ax4.set_facecolor(background_color)
ax4.tick_params(axis = "y", which = "both", left = False)
ax4_sns = sns.barplot(ax=ax4, x=temp_train.index, y=temp_train/1000, zorder=2)
ax4_sns.set_xlabel("feature_22",fontsize=5, weight='bold')
ax4_sns.set_ylabel('')
ax4.yaxis.set_major_formatter(ticker.PercentFormatter())
ax4_sns.grid(which='major', axis='x', zorder=0, color='#EEEEEE')
ax4_sns.grid(which='major', axis='y', zorder=0, color='#EEEEEE')
ax4_sns.tick_params(labelsize=5)


From the statistical summary, we know that the data range of Feature 38 and Feature 14 column is considerably larger than the other numeric columns.
Feature 38 and Feature 14 has median 1 and all the remaining columns median is zero.

In [ ]:
plt.rcParams['figure.dpi'] = 300
fig = plt.figure(figsize=(5, 2))
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])

ax0.tick_params(axis = "y", which = "both", left = False)

# KDE plots
ax0_sns = sns.kdeplot(ax=ax0, x=train_df['feature_14'], zorder=2, shade=True)
ax0_sns = sns.kdeplot(ax=ax0, x=test_df['feature_14'], zorder=2, shade=True)

# Axis and grid customization
ax0_sns.set_xlabel("feature_14",fontsize=5, weight='bold')
ax0_sns.set_ylabel('')
ax0.grid(which='major', axis='x', zorder=0, color='#EEEEEE')
ax0.grid(which='major', axis='y', zorder=0, color='#EEEEEE')

# Legend params
ax0.legend(['train', 'test'], prop={'size': 5})
ax0_sns.tick_params(labelsize=5)

plt.show()

In [ ]:
plt.rcParams['figure.dpi'] = 300
fig = plt.figure(figsize=(5, 2))
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])

ax0.tick_params(axis = "y", which = "both", left = False)

# KDE plots
ax0_sns = sns.kdeplot(ax=ax0, x=train_df['feature_38'], zorder=2, shade=True)
ax0_sns = sns.kdeplot(ax=ax0, x=test_df['feature_38'], zorder=2, shade=True)

# Axis and grid customization
ax0_sns.set_xlabel("feature_38",fontsize=5, weight='bold')
ax0_sns.set_ylabel('')
ax0.grid(which='major', axis='x', zorder=0, color='#EEEEEE')
ax0.grid(which='major', axis='y', zorder=0, color='#EEEEEE')

# Legend params
ax0.legend(['train', 'test'], prop={'size': 5})
ax0_sns.tick_params(labelsize=5)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

# x = [f'feature_{i}' for i in range(50)]
y = sorted([len(train_df[f'feature_{i}'].unique()) for i in range(50)])

ax.bar(range(50), y, zorder=10)
ax.set_xticks([])
ax.set_yticks(range(0, 80, 5))
ax.margins(0.02)

ax.set_title('TRAIN : # of Features Unique Values', loc='left', fontweight='bold')
ax.grid(axis='y', linestyle='--', zorder=5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

# x = [f'feature_{i}' for i in range(50)]
y = sorted([len(test_df[f'feature_{i}'].unique()) for i in range(50)])

ax.bar(range(50), y, zorder=10)
ax.set_xticks([])
ax.set_yticks(range(0, 80, 5))
ax.margins(0.02)

ax.set_title('TEST : # of Features Unique Values', loc='left', fontweight='bold')
ax.grid(axis='y', linestyle='--', zorder=5)
plt.show()

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")

train_p = train
lic = []
for col in train_p.columns[1:-1]:
    lic.append(col)
    
def plot(col):
    plt.figure(figsize = (18, 8))
    g = sns.countplot(x = col, hue = 'target', data = train_p)
    plt.legend(loc='upper right')
    plt.title("Distribution of "+ col,fontsize=15)
    plt.show();

for col in lic:
    plot(col)

# Model - CATBOOST

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv',index_col='id')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv',index_col='id')
submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv',index_col='id')

In [ ]:
# Let's assume all features are categorical. But some values exist only in train or test
# Clean it manually

test.loc[test['feature_3']==25,'feature_3']=26
test.loc[test['feature_4']==36,'feature_4']=37
test.loc[test['feature_21']==31,'feature_21']=36
test.loc[test['feature_25']==24,'feature_25']=23
test.loc[test['feature_34']==26,'feature_34']=25
test.loc[test['feature_49']==21,'feature_49']=20

print(len(train))
train = train[train['feature_5']!=10]

train = train[train['feature_6']!=26]
train = train[train['feature_6']!=27]

train = train[train['feature_7']!=30]
train = train[train['feature_7']!=31]

train = train[train['feature_9']!=17]

train = train[train['feature_10']!=16]

train = train[train['feature_11']!=12]

train = train[train['feature_15']!=20]

train = train[train['feature_16']!=18]

train = train[train['feature_23']!=18]
train = train[train['feature_23']!=19]

train = train[train['feature_27']!=29]

train = train[train['feature_28']!=23]

train = train[train['feature_29']!=13]

train = train[train['feature_33']!=24]

train = train[train['feature_32']!=26]
train = train[train['feature_32']!=27]

train = train[train['feature_35']!=43]
train = train[train['feature_35']!=-2]
train = train[train['feature_35']!=38]
train = train[train['feature_35']!=39]


train = train[train['feature_38']!=65]
train = train[train['feature_38']!=55]
train = train[train['feature_38']!=-8]
train = train[train['feature_38']!=-3]
train = train[train['feature_38']!=-2]
train = train[train['feature_38']!=63]

train = train[train['feature_39']!=65]
train = train[train['feature_39']!=66]
train = train[train['feature_39']!=-5]
train = train[train['feature_39']!=-3]
train = train[train['feature_39']!=-2]
train = train[train['feature_39']!=63]

train = train[train['feature_42']!=37]
train = train[train['feature_42']!=-2]
train = train[train['feature_42']!=-1]

train = train[train['feature_43']!=33]
train = train[train['feature_43']!=31]

print(len(train))

In [ ]:
columns = train.columns[:-1]
cat_features = ['feature_0',
                'feature_2',
                'feature_3',
                'feature_4',
                'feature_5',
                'feature_6',
                'feature_7',
                'feature_9',
                'feature_10',
                'feature_11',
                'feature_12',
                'feature_13',
                'feature_15',
                'feature_16',
                'feature_17',
                'feature_18',
                'feature_20',
                'feature_21',
                'feature_22',
                'feature_23',
                'feature_25',
                'feature_26',
                'feature_27',
                'feature_28',
                'feature_29',
                'feature_32',
                'feature_33',
                'feature_34',
                'feature_35',
                'feature_36',
                'feature_38',
                'feature_39',
                'feature_42',
                'feature_43',
                'feature_44',
                'feature_46',
                'feature_49']

In [ ]:
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
def train_models(df, columns, cat_features, random_state, n_splits):

    models = []
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True,
                            random_state=random_state)  # create folds
    X_train = df[columns]
    y_train = df['target']
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X_train,  y_train)):
        print('Fold {}/{}'.format(n_fold, n_splits))
        train_X, train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_X, valid_y = X_train.iloc[valid_idx], y_train.iloc[valid_idx]
        dataset = Pool(train_X, train_y, cat_features)
        evalset = Pool(valid_X, valid_y, cat_features)
        model = CatBoostClassifier(
            task_type="GPU",
            depth=4,
            max_ctr_complexity=15,
            # border_count=1024,
            iterations=50000,
            od_wait=1000, od_type='Iter',
            # l2_leaf_reg=0.01,
            learning_rate=0.01,
            min_data_in_leaf=1,
            use_best_model=True,
            loss_function='MultiClass'

        )
        model.fit(dataset, plot=False, verbose=500, eval_set=evalset)        
        _record = {
            'model': model,
        }
        models.append(_record)

    return models

In [ ]:
models = train_models(df=train, columns=columns, cat_features=cat_features, random_state=43, n_splits=9)

In [ ]:
c1_columns = []
c2_columns = []
c3_columns = []
c4_columns = []

for idx,m in enumerate(models):
    print(idx)
    _proba = m['model'].predict_proba(test[columns])
    
    c1_name = 'm{}_c1'.format(idx)
    c1_columns.append(c1_name)
    test[c1_name] = _proba[:,0]
    
    c2_name = 'm{}_c2'.format(idx)
    c2_columns.append(c2_name)
    test[c2_name] = _proba[:,1]
    
    c3_name = 'm{}_c3'.format(idx)
    c3_columns.append(c3_name)
    test[c3_name] = _proba[:,2]
    
    c4_name = 'm{}_c4'.format(idx)
    c4_columns.append(c4_name)
    test[c4_name] = _proba[:,3]

In [ ]:
test['Class_1'] = test[c1_columns].apply(np.mean,axis=1)
test['Class_2'] = test[c2_columns].apply(np.mean,axis=1)
test['Class_3'] = test[c3_columns].apply(np.mean,axis=1)
test['Class_4'] = test[c4_columns].apply(np.mean,axis=1)
test.to_csv('test3.csv')

In [ ]:
submission['Class_1'] = test['Class_1']
submission['Class_2'] = test['Class_2']
submission['Class_3'] = test['Class_3']
submission['Class_4'] = test['Class_4']
submission.to_csv('catboost-3.csv')

# Model Explainablity - SHAP

SHAP (SHapley Additive exPlanations) is a game theoretic approach to explain the output of any machine learning model. It connects optimal credit allocation with local explanations using the classic Shapley values from game theory and their related extensions (see papers for details and citations).


![SHAP](https://shap.readthedocs.io/en/latest/_images/shap_header.png)

FOR THIS I WILL BE USING A LGBM MODEL!! CHEERS! 

In [ ]:
import shap
import pandas as pd
import numpy as np
import seaborn as sns

from tqdm import tqdm
sns.set_style('whitegrid')
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv", index_col = 'id')
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv", index_col = 'id')
train = train[~train.drop('target', axis = 1).duplicated()]

X = pd.DataFrame(train.drop("target", axis = 1))

lencoder = LabelEncoder()
y = pd.DataFrame(lencoder.fit_transform(train['target']), columns=['target'])

In [ ]:
test_preds = None
train_rmse = 0
val_rmse = 0
n_splits = 10

model =  LGBMClassifier()

skf = StratifiedKFold(n_splits = n_splits, shuffle = True,  random_state = 0)

for tr_index , val_index in tqdm(skf.split(X.values , y.values), total=skf.get_n_splits(), desc="k-fold"):

    x_train_o, x_val_o = X.iloc[tr_index] , X.iloc[val_index]
    y_train_o, y_val_o = y.iloc[tr_index] , y.iloc[val_index]

    eval_set = [(x_val_o, y_val_o)]

    model.fit(x_train_o, y_train_o, eval_set = eval_set, early_stopping_rounds=100, verbose=False)

    train_preds = model.predict(x_train_o)
    train_rmse += mean_squared_error(y_train_o ,train_preds , squared = False)

    val_preds = model.predict(x_val_o)
    val_rmse += mean_squared_error(y_val_o , val_preds , squared = False)

    if test_preds is None:
        test_preds = model.predict_proba(test.values)
    else:
        test_preds += model.predict_proba(test.values)

    print(f"\nAverage Training RMSE : {train_rmse / n_splits}")
    print(f"Average Validation RMSE : {val_rmse / n_splits}\n")

    test_preds /= n_splits

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test)

In [ ]:
shap.summary_plot(shap_values, X)

**Observaitons**

- Feature_14 and feature_15 affect the model the most, but you can also see the balance between classes (feature_15 plays big role in class_2 prediction) - they are definitely important variables
- The most important feature for class_0 is feature_25, class_1 -> feature_14, class_2 -> feature_15, class_3 -> feature_31

In [ ]:
selected_features = ["feature_25", "feature_14", "feature_15", "feature_31"]

plt.figure(figsize=(20,5))
c = 1
for feat in selected_features:
    plt.subplot(1, 4, c)
    sns.histplot(x = feat, data = test, bins=10)
    c = c + 1    
plt.show()

**Observations:**

- Looks like most of values is ZERO(!) This is why you can observe some patterns later during class prediction analysis.
- This could be a potential problem with the model. Which will divide classes with respect to 0. And if there is a majority of them then ...


In [ ]:
test[selected_features].describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

In [ ]:
selected_features = ["feature_14", "feature_15", "feature_6", "feature_16", "feature_31", "feature_37"]

plt.figure(figsize=(15,10))
c = 1
for feat in selected_features:
    plt.subplot(2, 3, c)
    sns.histplot(x = feat, data = test, bins=10)
    c = c + 1    
plt.show()

In [ ]:
test[selected_features].describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

In [ ]:
shap.summary_plot(shap_values[0], test)

In [ ]:
selected_features = ["feature_25", "feature_37", "feature_23", "feature_38"]

plt.figure(figsize=(30,5))
c = 1
for feat in selected_features:
    plt.subplot(1, 4, c)
    sns.histplot(x = feat, data = test, bins=10)
    c = c + 1    
plt.show()

**Observations:**

- There is no obvious patters here, this is why model has problem with class 0 prediction - it works almost randomly (I think but I could be wrong)
as you can see reds are on the left and right - but we can see that most "high" values are on right side of chart
- almost all features are 0 balanced ... so 0 can drive model to Class 0 and .... rest of Class -> ZERO plays main role in this competition.
- Feature 25 - we can see that "medium" feature value (purple) drives model to predict class 0
- Feature 37 - the highest value (red) then class 0 is predicted,
- Feature 23 - high values drive model to predict class 0
- there is almost no uniform distribution

In [ ]:
shap.summary_plot(shap_values[1], test)

**Conclusion:**

- if most of features (till 20) are "low" they drives model to other class (not class 1, probably for class 3)
- feature 14 and "low" values says that probaly there is no Class_1
- feature 14->37 are uniform - "higher" values drive model to predict Class_1
- feature_34 - "higher" values drive model to Class_0

In [ ]:
selected_features = ["feature_14", "feature_6", "feature_28", "feature_37"]

plt.figure(figsize=(30,5))
c = 1
for feat in selected_features:
    plt.subplot(1, 4, c)
    sns.histplot(x = feat, data = test, bins=10)
    c = c + 1    
plt.show()

In [ ]:
 shap.summary_plot(shap_values[2], test)

**Conclusion:**

- The most important feature is 15 and 14. Here we can see obvious pattens. In feature_15 small values (probalby 0) drives model to Class 2. Feature_14 - high values drive to Class 2.
- Feature_2 there is seen good separation - all positive values (blue blob is 0) highly influence model in Class_2 direction.
- Almost in all features from list we can see that "high" values drives model to Class_2

In [ ]:
selected_features = ["feature_15", "feature_14", "feature_2", "feature_11"]

plt.figure(figsize=(30,5))
c = 1
for feat in selected_features:
    plt.subplot(1, 4, c)
    sns.histplot(x = feat, data = test, bins=10)
    c = c + 1    
plt.show()

In [ ]:
 shap.summary_plot(shap_values[3], test)

**Conclusion:**

- From feature 31 to 23 ... we can see that small values drive model to Class_3. This is probably 0 (we shold see data in these feature).
- The bigger number in features the less chance that Class 3 will be predicted.

In [ ]:
selected_features = ["feature_31", "feature_24", "feature_14", "feature_16"]

plt.figure(figsize=(30,5))
c = 1
for feat in selected_features:
    plt.subplot(1, 4, c)
    sns.histplot(x = feat, data = test, bins=10)
    c = c + 1    
plt.show()

References for the code : 

*model* : 
https://www.kaggle.com/belov38/catboost-catfeatures

*shap* : 
https://www.kaggle.com/remekkinas/shap-lgbm-looking-for-best-features

### Thank you for going through the plots I have created as of now! 

### I will keep adding more and try to do a more detailed EDA this time! 
**Do upvote and check out my other kernels!!**

***Follow me for more content!!!***

Some of my other works:
- [**TPS APRIL**](https://www.kaggle.com/udbhavpangotra/tps-apr21-eda-model )
- [**Heart Attacks!!! Find out what factors influence them the most!** ](https://www.kaggle.com/udbhavpangotra/heart-attacks-extensive-eda-and-visualizations) 
- [**What are the Birtish watching on Youtube!**](https://www.kaggle.com/udbhavpangotra/what-do-people-use-youtube-for-in-great-britain)